# Laboratorio 1 - Estructuras de Datos II

In [1]:
import pandas as pd
import seaborn as sns
df = pd.read_csv("co_properties_final.csv")
df.head()
#Trabajen aquí, gracias.

,title,department,city,property_type,latitude,longitude,surface_total,surface_covered,bedrooms,bathrooms,operation_type,price
0,Apartamento En Arriendo/venta En Barranquilla ...,Atlántico,Barranquilla,Apartamento,11.013,-74.836,70.0,70.0,1.0,2.0,Venta,250000000.0
1,Casa En Venta En Cali Ciudad Jardn Cod. VVLZ3039,Valle del Cauca,Cali,Casa,3.368,-76.531,420.0,420.0,5.0,5.0,Venta,950000000.0
2,Casa En Arriendo/venta En Barranquilla Bellavi...,Atlántico,Barranquilla,Casa,10.997,-74.791,83.0,150.0,3.0,3.0,Venta,320000000.0
3,Casa En Arriendo En Chia Chia Cod. AINH2992,Cundinamarca,Chía,Casa,4.845,-74.057,580.0,2.0,8.0,3.0,Arriendo,8000000.0
4,Apartamento En Arriendo En Cali Ciudad Crdoba ...,Valle del Cauca,Cali,Apartamento,3.402,-76.507,56.0,56.0,3.0,1.0,Arriendo,550000.0
